In [1]:
import os.path as path
import glob, os
from datetime import datetime
from importlib.metadata import version
import cobra
import thermo_flux
from thermo_flux.io import load_excel as ex
from thermo_flux.io import helper_load as hl
from thermo_flux.core.model import ThermoModel
from thermo_flux.solver.gurobi import variability_analysis, variability_results, compute_IIS
from equilibrator_api import  Q_
import pandas as pd
import numpy as np
from scripts.logger import write_to_log
import gurobipy as gp
from gurobipy import GRB
from scripts.gen_model import gen_model
from scripts.gen_model import apply_physio_data
from scripts.gen_model import constrain_bounds_fva
from scripts.reaction_utils import list_and_remove_blocked_reactions
from scripts.reaction_utils import list_blocked_reactions
from scripts.reaction_utils import count_blocked_pathways
from scripts.reaction_utils import tfva_write_scenarios
import enkie
import pta
import importlib

pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


NameError: name 'Model' is not defined

In [2]:
INPUT_MODEL = "datafiles/model.xlsx"
INPUT_KEGGS = "datafiles/ecoli_kegg_id.csv"
INPUT_REED = "regression/reed.csv"
INPUT_INCHI = "regression/InChIs.csv"
INPUT_GAMS = "regression/model_Ecoli_from-gams.xlsx"
INPUT_EXP_DATA = "regression/allPhysioData_formatted_forGSM_20230831.csv"
INPUT_EXP_CONC = "regression/allConcRange_20230912.csv"
INPUT_METABOLOMICS = "regression/metabolomics-Kochanowski_20230925.csv"

MODEL_NAME = "ecoli"

OUTPUT_DIR = "output"
OUTPUT_NAME = f"pta_test"
OUTPUT_LOG = f"{OUTPUT_DIR}{path.sep}{OUTPUT_NAME}_log.txt"

CONDITIONS_TO_REGRESS = ["WT-Glc_I"]

INCLUDE_CO2 = True
INCLUDE_O2 = True
ALLOW_OTHER_EXCRETION = False
RELAX_EXP_FLUX_BOUNDS = 2.0

time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
write_to_log(OUTPUT_LOG, f"Started analysis at: {time}", "w")

# Write package versions:
modules = ["pandas", "numpy", "equilibrator_api", "cobra"]
write_to_log(OUTPUT_LOG, f"Package versions used:")
versions_packages = [f"  {m}: {version(m)}\n" for m in modules]
write_to_log(OUTPUT_LOG, "".join(versions_packages))

In [ ]:
import os
print(os.environ.get('GUROBI_HOME'))
print(os.environ.get('LD_LIBRARY_PATH'))


In [ ]:
model = pta.load_example_model("e_coli_core")
model.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = 0.5

In [3]:
tmodel = gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, OUTPUT_LOG, True, True)
tmodel = apply_physio_data(tmodel, CONDITIONS_TO_REGRESS[0], INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, False, OUTPUT_LOG)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2731723
Academic license 2731723 - for non-commercial use only - registered to l.___@student.rug.nl
['Parameters', 'Exchange reactions', 'Reactions', 'Biomass Composition', 'Transmembrane reactions', 'Metabolites', 'references', 'Transmembrane_reactions_reed', 'Transmembrane reactions_Orth', 'Transmembrane reactions old', 'Sheet3', 'log', 'subsystems']
*** Reading data from Reactions ***
unknown metabolite '2dhglcn[c]' created
unknown metabolite 'nadh[c]' created
unknown metabolite 'glcn[c]' created
unknown metabolite 'nad[c]' created
unknown metabolite 'nadph[c]' created
unknown metabolite 'nadp[c]' created
unknown metabolite '2dhguln[c]' created
unknown metabolite 'idon-L[c]' created
unknown metabolite '3hcinnm[c]' created
unknown metabolite 'o2[c]' created
unknown metabolite 'dhcinnm[c]' created
unknown metabolite 'h2o[c]' created
unknown metabolite '3hpppn[c]' created
unknown metabolite 'dhpppn[c]' c

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


etoh_c NOTHING DONE!
gdp_c NOTHING DONE!
itp_c NOTHING DONE!
idp_c NOTHING DONE!
ametam_c NOTHING DONE!
camp_c NOTHING DONE!
adocbi_c NOTHING DONE!
rdmbzi_c NOTHING DONE!
adocbl_c NOTHING DONE!
gmp_c NOTHING DONE!
prpp_c NOTHING DONE!
aps_c NOTHING DONE!
paps_c NOTHING DONE!
dcamp_c NOTHING DONE!
fum_c NOTHING DONE!
25aics_c NOTHING DONE!
aicar_c NOTHING DONE!
asp-L_c NOTHING DONE!
imp_c NOTHING DONE!
acgam6p_c NOTHING DONE!
gam6p_c NOTHING DONE!
adphep-DD_c NOTHING DONE!
adphep-LD_c NOTHING DONE!
agm_c NOTHING DONE!
ptrc_c NOTHING DONE!
urea_c NOTHING DONE!
hcys-L_c NOTHING DONE!
rhcys_c NOTHING DONE!
10fthf_c NOTHING DONE!
fprica_c NOTHING DONE!
thf_c NOTHING DONE!
air_c NOTHING DONE!
5caiz_c NOTHING DONE!
5aizc_c NOTHING DONE!
ala-D_c NOTHING DONE!
alaala_c NOTHING DONE!
ala-L_c NOTHING DONE!
pydx5p_c NOTHING DONE!
pyam5p_c NOTHING DONE!
glyald_c NOTHING DONE!
glyc_c NOTHING DONE!
pacald_c NOTHING DONE!
pac_c NOTHING DONE!
alltt_c NOTHING DONE!
urdglyc_c NOTHING DONE!
alltn_c NOTHIN

In [4]:
blocked_p = list_blocked_reactions(tmodel, CONDITIONS_TO_REGRESS[0], OUTPUT_LOG, 1, False)
print(len(blocked_p))

tmodel.remove_reactions(blocked_p, remove_orphans=True)
for rxn in tmodel.reactions:
    thermo_flux.tools.drg_tools.reaction_balance(rxn, balance_charge=True, balance_mg=False)
tmodel.update_thermo_info(fit_unknown_dfG0=True)

['3HCINNMH', '3HPPPNH', '4HTHRS', '5DGLCNR', 'AADDGT', 'AB6PGH', 'ACACCT', 'ACACT1r', 'ACBIPGT', 'ACGAMT', 'ACMAMUT', 'ACNML', 'ACONMT', 'ACPS1', 'ADHEr', 'ADK4', 'ADNCYC', 'ADOCBIK', 'ADOCBLS', 'AGDC', 'AGMT', 'AHC', 'AHCYSNS', 'ALCD19', 'ALDD19x', 'ALLTAH', 'alltn', 'ALTRH', 'AMANAPE', 'AMANK', 'AMAOTr', 'AMMQT82', 'AMPMS', 'AOXSr', 'AP4AH', 'AP5AH', 'ARAI', 'ARGDC', 'BTS2', 'BUTCT', 'CBIAT', 'CBLAT', 'CDPMEK', 'CHRPL', 'CINNDO', 'CPPPGO', 'CRNBTCT', 'CRNCBCT', 'CRNCDH', 'CYANST', 'CYNTAH', 'DBTSr', 'DDGALK', 'DDGLK', 'DDPGALA', 'DHBD', 'DHBSr', 'DHCIND', 'DHCINDO', 'DHNAOT', 'DHPPD', 'DHPTDC', 'DMATT', 'DMQMT', 'DOGULNR', 'DXPRIi', 'DXPS', 'DXYLK', 'E4PD', 'ECAPEC', 'EDTXS3', 'EDTXS4', 'ENTCS', 'FAO3', 'FAO4', 'FCI', 'FCLK', 'FCLPA', 'FCLT', 'FFSD', 'FHL', 'G1PTT', 'G1SATi', 'GALCTD', 'GALCTND', 'GALKr', 'GALS3', 'GDMANE', 'GLCRAL', 'GLCRD', 'GLTPD', 'glucys', 'GLUTRR', 'GLUTRS', 'GMAND', 'GOFUCR', 'GP4GH', 'GPDDA1', 'GPDDA3', 'GPDDA5', 'GRTT', 'GTHOr', 'GTHS', 'GUI1', 'GUI2', 'HBZO

In [54]:
for r in tmodel.reactions:
    if(r.upper_bound <-9):
        print(r.id)
        print(r.lower_bound)
        print(r.upper_bound)

EX_glc
-12.148855548767477
-9.302806592838524
EX_o2
-25.64184695305984
-9.117049900823162


In [55]:
import cobra.io
from scripts.io import export_tmodel_cobra
from scripts.io import load_tmodel_cobra
from scripts.metabolite_utils import remove_orphan_metabolites

output_path = export_tmodel_cobra(tmodel, "models/pta_test.json")

cobra_model = load_tmodel_cobra(output_path, strip_compartment=True, metabolite_namespace= "bigg.metabolite")

remove_orphan_metabolites(cobra_model)
pta.prepare_for_pta(cobra_model, biomass_id="biomass_EX", atpm_id="ATPHYD", remove_blocked_reactions=False, tighten_bounds=False, prevent_loops=False, autoresolve_inconsistencies=False)

[{'id': '2dhglcn_c', 'name': '2dhglcn_c', 'compartment': 'c', 'charge': -1, 'formula': 'H8'}, {'id': 'nadh_c', 'name': 'nadh_c', 'compartment': 'c', 'charge': -2, 'formula': 'H27'}, {'id': 'glcn_c', 'name': 'glcn_c', 'compartment': 'c', 'charge': -1, 'formula': 'H11'}, {'id': 'nad_c', 'name': 'nad_c', 'compartment': 'c', 'charge': -1, 'formula': 'H26'}, {'id': 'nadph_c', 'name': 'nadph_c', 'compartment': 'c', 'charge': -4, 'formula': 'H26'}, {'id': 'nadp_c', 'name': 'nadp_c', 'compartment': 'c', 'charge': -3, 'formula': 'H25'}, {'id': '2dhguln_c', 'name': '2dhguln_c', 'compartment': 'c', 'charge': -1, 'formula': 'H9'}, {'id': 'idon-L_c', 'name': 'idon-L_c', 'compartment': 'c', 'charge': -1, 'formula': 'H11'}, {'id': 'o2_c', 'name': 'o2_c', 'compartment': 'c', 'charge': 0, 'formula': 'H0'}, {'id': 'h2o_c', 'name': 'h2o_c', 'compartment': 'c', 'charge': 0, 'formula': 'H2'}, {'id': 'pi_c', 'name': 'pi_c', 'compartment': 'c', 'charge': -2, 'formula': 'H1'}, {'id': '5dglcn_c', 'name': '5dgl

In [56]:
from pta import StructuralAssessment
from scripts.reaction_utils import fix_negative_upper_bounds

temp_model = cobra_model.copy()
fix_negative_upper_bounds(temp_model)
assessment = StructuralAssessment(temp_model, "biomass_EX", "ATPHYD")

ImportError: cannot import name 'fix_negative_upper_bounds' from 'scripts.reaction_utils' (/home/laurens/MSC-Thermo/scripts/reaction_utils.py)

In [34]:
atp_rxns = [x for x in cobra_model.reactions if "ATP" in x.id]
print(atp_rxns)
print(cobra_model.reactions.ATPHYD)

bio_rxns = [x for x in cobra_model.reactions if "biomass" in x.id]
print(bio_rxns)
for b in bio_rxns:
    print(b)

[<Reaction ATPHYD at 0x755984804b50>, <Reaction ATPPRT at 0x755984805210>, <Reaction PRATPP at 0x7559615a5dd0>, <Reaction ATPS4r at 0x7559615cbb90>]
ATPHYD: atp_c + h2o_c --> adp_c + h_c + pi_c
[<Reaction biomass at 0x7559615e1450>, <Reaction biomass_ce at 0x7559867356d0>, <Reaction biomass_EX at 0x755986735a10>]
biomass: 0.05 5mthf_c + 5e-05 accoa_c + 0.488 ala-L_c + 0.001 amp_c + 0.281 arg-L_c + 0.229 asn-L_c + 0.229 asp-L_c + 54.1352 atp_c + 0.000129 clpnEC_c + 6e-06 coa_c + 0.126 ctp_c + 0.087 cys-L_c + 0.0247 datp_c + 0.0254 dctp_c + 0.0254 dgtp_c + 0.0247 dttp_c + 1e-05 fad_c + 0.25 gln-L_c + 0.25 glu-L_c + 0.582 gly_c + 0.154 glycogen_c + 0.203 gtp_c + 53.95 h2o_c + 0.09 his-L_c + 0.276 ile-L_c + 0.428 leu-L_c + 0.0084 lpsEC_c + 0.326 lys-L_c + 0.146 met-L_c + 0.00215 nad_c + 5e-05 nadh_c + 0.00013 nadp_c + 0.0004 nadph_c + 0.001935 peEC_c + 0.0276 peptidoEC_c + 0.000464 pgEC_c + 0.176 phe-L_c + 0.21 pro-L_c + 5.2e-05 psEC_c + 0.035 ptrc_c + 0.205 ser-L_c + 0.007 spmd_c + 3e-06 

In [7]:
print([x.annotation for x in cobra_model.metabolites])

[{'bigg.metabolite': '2dhglcn'}, {'bigg.metabolite': 'nadh'}, {'bigg.metabolite': 'glcn'}, {'bigg.metabolite': 'nad'}, {'bigg.metabolite': 'nadph'}, {'bigg.metabolite': 'nadp'}, {'bigg.metabolite': '2dhguln'}, {'bigg.metabolite': 'idon__L'}, {'bigg.metabolite': 'o2'}, {'bigg.metabolite': 'h2o'}, {'bigg.metabolite': 'pi'}, {'bigg.metabolite': '5dglcn'}, {'bigg.metabolite': 'ru5p__D'}, {'bigg.metabolite': 'ara5p'}, {'bigg.metabolite': 'ACP'}, {'bigg.metabolite': 'atp'}, {'bigg.metabolite': 'ttdca'}, {'bigg.metabolite': 'amp'}, {'bigg.metabolite': 'myrsACP'}, {'bigg.metabolite': 'ppi'}, {'bigg.metabolite': 'ttdcea'}, {'bigg.metabolite': 'tdeACP'}, {'bigg.metabolite': 'hdca'}, {'bigg.metabolite': 'palmACP'}, {'bigg.metabolite': 'hdcea'}, {'bigg.metabolite': 'hdeACP'}, {'bigg.metabolite': 'ocdcea'}, {'bigg.metabolite': 'octeACP'}, {'bigg.metabolite': 'dtdp'}, {'bigg.metabolite': 'g6p'}, {'bigg.metabolite': '4abut'}, {'bigg.metabolite': 'akg'}, {'bigg.metabolite': 'glu__L'}, {'bigg.metabolit

In [6]:
from scripts.reaction_utils import pta_get_concentrations

In [7]:
from scripts.reaction_utils import pta_get_parameters

In [46]:
solution = cobra_model.optimize()
print(solution.status)

optimal


In [47]:
met_namespace = "bigg.metabolite"

parameters = pta_get_parameters(tmodel) # Use tmodel for parameters as this info is lost in the normal cobra model
concentrations = pta_get_concentrations(tmodel, met_namespace) #

thermodynamic_space = pta.ThermodynamicSpace.from_cobrapy_model(
    cobra_model,
    metabolites_namespace = met_namespace,
    parameters=parameters,
)

    bigg.metabolite:2dhguln
    bigg.metabolite:ACP
    bigg.metabolite:ttdca
    bigg.metabolite:myrsACP
    bigg.metabolite:ttdcea
    bigg.metabolite:tdeACP
    bigg.metabolite:palmACP
    bigg.metabolite:hdcea
    bigg.metabolite:hdeACP
    bigg.metabolite:octeACP
    bigg.metabolite:co2tot
    kegg.compound:C00163
    bigg.metabolite:acACP
    bigg.metabolite:actACP
    bigg.metabolite:malACP
    bigg.metabolite:ddcaACP
    bigg.metabolite:cdpdag1
    bigg.metabolite:pa_EC
    bigg.metabolite:pg_EC
    bigg.metabolite:clpn_EC
    bigg.metabolite:12dgr_EC
    __unreliable__:bigg.metabolite:glycogen
    bigg.metabolite:3hmrsACP
    bigg.metabolite:agpg_EC
    bigg.metabolite:agpe_EC
    bigg.metabolite:agpc_EC
    bigg.metabolite:apg_EC
    bigg.metabolite:lps_EC
    bigg.metabolite:micit
    bigg.metabolite:pe_EC
    bigg.metabolite:pgp_EC
    bigg.metabolite:peptido_EC
    bigg.metabolite:ps_EC
    bigg.metabolite:charge
    bigg.metabolite:co2tot
    bigg.metabolite:charge
Using 

In [48]:
problem = pta.PmoProblem(cobra_model, thermodynamic_space, solver='GUROBI' )
problem.solve(verbose=True)

(CVXPY) Nov 26 03:38:01 PM: Your problem has 1967 variables, 4137 constraints, and 0 parameters.
(CVXPY) Nov 26 03:38:01 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 26 03:38:01 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 26 03:38:01 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 26 03:38:01 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Nov 26 03:38:01 PM: Compiling problem (target solver=GUROBI).
(CVXPY) Nov 26 03:38:01 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> GUROBI
(CVXPY) Nov 26 03:38:01 PM: Applying reduction Dcp2Cone
(CVXPY) Nov 26 03:38:01 PM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 26 03:38:01 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Nov 26 03:38:01 PM: Applying reduction GUROBI


                                     CVXPY                                     
                                     v1.7.3                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------


(CVXPY) Nov 26 03:38:01 PM: Finished problem compilation (took 2.003e-01 seconds).
(CVXPY) Nov 26 03:38:01 PM: Invoking solver GUROBI  to obtain a solution.


-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
Set parameter OutputFlag to value 1
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter QCPDual to value 1
Set parameter IntFeasTol to value 1e-09
Set parameter FeasibilityTol to value 1e-09
Set parameter BarQCPConvTol to value 1e-08
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
FeasibilityTol  1e-09
IntFeasTol  1e-09
BarQCPConvTol  1e-08
QCPDual  1

Academic license 2731723 - for non-commercial use only - registered to l.___@student.rug.nl
Optimize a model with 5611 rows, 3443 columns a


    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
(CVXPY) Nov 26 03:46:46 PM: Problem status: infeasible_or_unbounded
(CVXPY) Nov 26 03:46:46 PM: Optimal value: nan
(CVXPY) Nov 26 03:46:46 PM: Compilation took 2.003e-01 seconds
(CVXPY) Nov 26 03:46:46 PM: Solver (including time spent in interface) took 5.254e+02 seconds


-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------


'infeasible_or_unbounded'

In [15]:
assessment = pta.QuantitativeAssessment(problem)
assessment.summary()

TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [ ]:
from pta.sampling.tfs import (
    TFSModel,
    sample_drg,
    sample_drg0_from_drg,
    sample_log_conc_from_drg,
    sample_fluxes_from_drg
)

In [ ]:
tfs_model = TFSModel(cobra_model, thermodynamic_space, solver="GUROBI")
result = sample_drg(tfs_model)